In [6]:
from google.colab import files
import pandas as pd
from google.colab import drive
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [12]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
uploaded = files.upload()

KeyboardInterrupt: 

In [4]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

NameError: name 'uploaded' is not defined

In [6]:
spam_ham_df = pd.read_csv('spam_ham_dataset.csv')
phising_df = pd.read_csv('Phishing_Email.csv')

In [8]:
phising_df

,Unnamed: 0,Email Text,Email Type
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,1,the other side of * galicismos * * galicismo *...,Safe Email
2,2,re : equistar deal tickets are you still avail...,Safe Email
3,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email
...,...,...,...
18645,18646,date a lonely housewife always wanted to date ...,Phishing Email
18646,18647,request submitted : access request for anita ....,Safe Email
18647,18648,"re : important - prc mtg hi dorn & john , as y...",Safe Email
18648,18649,press clippings - letter on californian utilit...,Safe Email


In [7]:
spam_ham_df

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [10]:
# select, rename, and copy df
phising_df_selected = phising_df[['Email Text', 'Email Type']].copy()
spam_ham_df_selected = spam_ham_df[['text', 'label']].rename(columns={'text': 'Email Text', 'label': 'Email Type'}).copy()

# change column names to match
phising_df_selected['Email Type'] = phising_df_selected['Email Type'].replace({'Phishing Email': 'Malicious Email'})
spam_ham_df_selected['Email Type'] = spam_ham_df_selected['Email Type'].replace({'ham': 'Safe Email', 'spam': 'Malicious Email'})

# append and create new df
combined_df = pd.concat([phising_df_selected, spam_ham_df_selected], ignore_index=True)

In [19]:
combined_df

,Email Text,Email Type
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,the other side of * galicismos * * galicismo *...,Safe Email
2,re : equistar deal tickets are you still avail...,Safe Email
3,\nHello I am your hot lil horny toy.\n I am...,Malicious Email
4,software at incredibly low prices ( 86 % lower...,Malicious Email
...,...,...
23816,Subject: put the 10 on the ft\r\nthe transport...,Safe Email
23817,Subject: 3 / 4 / 2000 and following noms\r\nhp...,Safe Email
23818,Subject: calpine daily gas nomination\r\n>\r\n...,Safe Email
23819,Subject: industrial worksheets for august 2000...,Safe Email


In [ ]:
def preprocess_text(text):
    # Cleaning: remove special characters and extra spaces
    # text = re.sub(r'[\t\n\r]+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s.,;\'\":-]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Stop Words Removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]

    # Rejoin tokens into a single string
    return ' '.join(tokens)

combined_df['Email Text'] = combined_df['Email Text'].apply(preprocess_text)

In [8]:
drive.mount('/content/drive')
#combined_df.to_csv('/content/drive/My Drive/combined_emails.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# prompt: find cell with "Administration would be acting quite differently from the Bush "

cell = combined_df[combined_df['Email Text'] == 'Administration would be acting quite differently from the Bush ']
print(cell)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [24]:
combined_df.to_pickle('/content/drive/My Drive/combined_df.pkl')

In [9]:
import pickle
DATA_PATH = "/content/drive/My Drive"
infile = open(DATA_PATH+'/combined_df.pkl','rb')
dfdf = pickle.load(infile)

In [10]:
dfdf

,Email Text,Email Type
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,the other side of * galicismos * * galicismo *...,Safe Email
2,re : equistar deal tickets are you still avail...,Safe Email
3,\nHello I am your hot lil horny toy.\n I am...,Malicious Email
4,software at incredibly low prices ( 86 % lower...,Malicious Email
...,...,...
23816,Subject: put the 10 on the ft\r\nthe transport...,Safe Email
23817,Subject: 3 / 4 / 2000 and following noms\r\nhp...,Safe Email
23818,Subject: calpine daily gas nomination\r\n>\r\n...,Safe Email
23819,Subject: industrial worksheets for august 2000...,Safe Email


In [14]:
unique_types = dfdf['Email Text'].apply(type).unique()
print(unique_types)

# Find rows where 'email_text' is not a string
non_string_entries = dfdf[dfdf['Email Text'].apply(lambda x: not isinstance(x, str))]

# Display these rows to understand what they contain
print(non_string_entries)

[<class 'str'> <class 'float'>]
      Email Text       Email Type
31           NaN  Malicious Email
387          NaN  Malicious Email
1883         NaN  Malicious Email
2049         NaN  Malicious Email
2451         NaN  Malicious Email
2972         NaN  Malicious Email
3627         NaN  Malicious Email
3806         NaN  Malicious Email
5763         NaN  Malicious Email
6299         NaN  Malicious Email
6821         NaN  Malicious Email
8594         NaN  Malicious Email
9999         NaN  Malicious Email
11069        NaN  Malicious Email
11320        NaN  Malicious Email
13843        NaN  Malicious Email


In [15]:
# remove NaNs
dfdf = dfdf.dropna(subset=['Email Text'])

In [16]:
def preprocess_text(text):
    # Cleaning: remove special characters and extra spaces
    # text = re.sub(r'[\t\n\r]+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s.,;\'\":-]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Stop Words Removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]

    # Rejoin tokens into a single string
    return ' '.join(tokens)

dfdf['Email Text'] = dfdf['Email Text'].apply(preprocess_text)

<ipython-input-16-385d4879c7eb>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfdf['Email Text'] = dfdf['Email Text'].apply(preprocess_text)


In [17]:
dfdf.to_pickle('/content/drive/My Drive/combined_cleaned.pkl')